In [7]:
import requests
from bs4 import BeautifulSoup
import json

from bs4 import BeautifulSoup
import re

In [8]:
import psycopg2
from psycopg2 import sql

def connect_to_db():
    """Establish a connection to the PostgreSQL database."""
    try:
        connection = psycopg2.connect(
            host="localhost",         # Replace with your database host
            database="postgres", # Replace with your database name
            user="postgres",     # Replace with your database user
            password="postgres"  # Replace with your database password
        )
        return connection
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error connecting to the database: {error}")
        return None

def create_table():
    """Create a table in the PostgreSQL database."""
    connection = connect_to_db()
    if connection is not None:
        try:
            cursor = connection.cursor()
            create_table_query = """
            CREATE TABLE IF NOT EXISTS cost_of_living (
                id SERIAL PRIMARY KEY,  -- Unique identifier for each record
                country VARCHAR(255),   -- Country name
                city VARCHAR(255),      -- City name (can be NULL if not applicable)
                category VARCHAR(255),  -- Category of the cost (e.g., Food, Transport)
                item VARCHAR(255),      -- Item name
                price VARCHAR(255)      -- Price of the item (stored as text for simplicity)
            );
            """
            cursor.execute(create_table_query)
            connection.commit()
            print("Table 'cost_of_living' created successfully.")
        except (Exception, psycopg2.DatabaseError) as error:
            print(f"Error creating table: {error}")
        finally:
            cursor.close()
            connection.close()
    else:
        print("Failed to connect to the database.")



In [2]:

if __name__ == "__main__":
    create_table()

Table 'cost_of_living' created successfully.


In [9]:
def connect_to_db():
    try:
        connection = psycopg2.connect(
            host="localhost",         # Replace with your database host
            database="postgres", # Replace with your database name
            user="postgres",     # Replace with your database user
            password="postgres"  # Replace with your database password
        )
        return connection
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error connecting to database: {error}")
        return None


In [10]:
class Extract_table:
    """ Extracts the table from the beautiful soup html page"""
    def __init__(self, page, country, city=None):
        self.Data = {}
        self.Table = page.find("table", {'class': 'data_wide_table'})
        self.country = country
        self.city = city

    def clean_string(self, s):
        s = re.sub(r"\u00a0", " ", s)
        s = re.sub(r"\n+", "", s)
        return s.strip()

    def extract(self):
        if not self.Table:
            return None

        key = None
        for row in self.Table("tr"):
            if row("th"):
                key = self.clean_string(row("th")[0].text)
                self.Data[key] = []
            elif key:
                cleaned_row = [self.clean_string(cell.text) for cell in row("td")]
                self.Data[key].append(cleaned_row)
                # Insert into the PostgreSQL database
                self.insert_into_db(key, cleaned_row)

        return self.Data

    def insert_into_db(self, category, row):
        connection = connect_to_db()
        if connection:
            cursor = connection.cursor()
            query = """
            INSERT INTO cost_of_living (country, city, category, item, price)
            VALUES (%s, %s, %s, %s, %s)
            """
            city_value = self.city if self.city else "N/A"
            cursor.execute(query, (self.country, city_value, category, row[0], row[1]))
            connection.commit()
            cursor.close()
            connection.close()


In [11]:
class API(object):
    """API to get a country"""
    def __init__(self, BASE_URL, Country, city=0):
        self.base = BASE_URL
        self.url = BASE_URL + "country_result.jsp?country=" + Country + "&displayCurrency=USD"
        self.country = Country
        self.result = {}
        response = self.get_page(self.url)
        if response:
            self.page = BeautifulSoup(response.text, "html.parser")
            self.get_city()
            EX = Extract_table(self.page, Country)
            self.result[Country] = EX.extract()
            if city:
                self.get_all_city()
        else:
            self.page = None
            self.city = None
            
    def get_result(self):
        """returns the result for the country"""
        return self.result[self.country]

    def get_page(self, url):
        """ get the page from the url"""
        request = requests.get(url)
        if request.status_code != 200:
            return None
        return request

    def get_city(self):
        """get all the city"""
        self.city_form = self.page.find("form", {"class": "standard_margin"})
        self.city = [values["value"] for values in self.city_form("option")]

    def get_single_city(self, city):
        """ get table with the city name for the country"""
        country = self.country
        city_page = self.base+"/city_result.jsp?country="+country+"&city="+city+"&displayCurrency=USD"
        self.city_page = BeautifulSoup(self.get_page(city_page).text, "html")
        table = Extract_table(self.city_page)
        return table.extract()

    def get_all_city(self):
        """ get the table of all the city and returns as a dict"""
        country = self.country
        self.result[country]["child"] = {}
        for city in self.city:
            print( "crawling Country -> %s, city -> %s"%(country, city))
            self.result[country]["child"][city] = self.get_single_city(city)

In [18]:
import requests
from bs4 import BeautifulSoup
import psycopg2
import re

class Extract_table:
    """Extracts the table from the BeautifulSoup HTML page and inserts into PostgreSQL"""
    
    def __init__(self, page, country, city=None):
        self.Table = page.find("table", {'class': 'data_wide_table'})  # Finding table with class name
        self.country = country
        self.city = city

    def clean_string(self, s):
        """Cleans unwanted characters from a string"""
        s = re.sub(r"\u00a0", " ", s)  # Replace non-breaking space with a regular space
        s = re.sub(r"\n+", "", s)      # Remove newline characters
        return s.strip()               # Remove any leading or trailing whitespace

    def extract(self):
        """Extracts the table contents and stores them to a PostgreSQL database"""
        if not self.Table:
            return None  # Return None if the table is not found

        connection = connect_to_db()
        if not connection:
            print("Failed to connect to the database.")
            return

        cursor = connection.cursor()
        key = None
        for row in self.Table("tr"):
            if row("th"):
                key = self.clean_string(row("th")[0].text)
            elif key:
                cleaned_row = [self.clean_string(cell.text) for cell in row("td")]
                self.insert_into_db(connection, cursor, key, cleaned_row)
        
        cursor.close()
        connection.close()

    def insert_into_db(self, connection, cursor, category, row):
        """Insert extracted data into the PostgreSQL database"""
        query = """
        INSERT INTO cost_of_living (country, city, category, item, price)
        VALUES (%s, %s, %s, %s, %s)
        """
        city_value = self.city if self.city else "N/A"
        cursor.execute(query, (self.country, city_value, category, row[0], row[1]))
        connection.commit()

def connect_to_db():
    """Establish a connection to the PostgreSQL database."""
    try:
        connection = psycopg2.connect(
            host="localhost",         # Replace with your database host
            database="postgres", # Replace with your database name
            user="postgres",     # Replace with your database user
            password="postgres"  # Replace with your database password
        )
        return connection
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error connecting to the database: {error}")
        return None

class API(object):
    """API to get a country and optionally its cities"""
    
    def __init__(self, BASE_URL, Country, city=0):
        self.base = BASE_URL
        self.url = BASE_URL + "country_result.jsp?country=" + Country + "&displayCurrency=USD"
        self.country = Country
        self.city = None
        response = self.get_page(self.url)
        if response:
            self.page = BeautifulSoup(response.text, "html.parser")
            self.get_city()
            EX = Extract_table(self.page, Country)
            EX.extract()
            if city:
                self.get_all_city()
        else:
            self.page = None
            self.city = None

    def get_page(self, url):
        """Get the page from the URL"""
        try:
            request = requests.get(url)
            if request.status_code != 200:
                return None
            return request
        except requests.RequestException as e:
            print(f"Request failed: {e}")
            return None

    def get_city(self):
        """Get all cities for a given country"""
        self.city_form = self.page.find("form", {"class": "standard_margin"})
        if self.city_form:
            self.city = [values["value"] for values in self.city_form("option")]

    def get_single_city(self, city):
        """Get table with the city name for the country"""
        city_page_url = f"{self.base}city_result.jsp?country={self.country}&city={city}&displayCurrency=USD"
        city_page = self.get_page(city_page_url)
        if city_page:
            city_soup = BeautifulSoup(city_page.text, "html.parser")
            table = Extract_table(city_soup, self.country, city)
            table.extract()
        else:
            print(f"Failed to retrieve data for city: {city}")

    def get_all_city(self):
        """Get the table of all the cities and store as a dict"""
        if not self.city:
            print(f"No cities found for country: {self.country}")
            return

        for city in self.city:
            print(f"Crawling Country -> {self.country}, City -> {city}")
            self.get_single_city(city)

# Example usage
if __name__ == "__main__":
    BASE_URL = "https://www.numbeo.com/cost-of-living/"
    country = "Western Sahara"
    api = API(BASE_URL, country, city=1)  # Set city=1 to include cities


Crawling Country -> Western Sahara, City -> 
Crawling Country -> Western Sahara, City -> El Aaiun
